# csv Agent로 자동으로 일정 찾기

In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_csv_agent

In [2]:
agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    "../data/cal_1.csv",
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [5]:
res = agent.run("이번 주 토요일 일정 알려줘, 응답 결과는 json 형식으로 출력해줘")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\nimport datetime\n\n# Filter rows for this week's Saturday\ntoday = datetime.date.today()\nsaturday = today + datetime.timedelta((5-today.weekday()) % 7)\nfiltered_df = df[df['calendar_start_date'].dt.date == saturday]\n\n# Convert dataframe to JSON\njson_result = filtered_df.to_json(orient='records')\njson_result"}`


[{"userId":1,"calendar_title":"3\uc8fc \ud68c\uc2dd","calendar_place":"\uc11c\uba74","calendar_start_date":1700265600000,"calendar_end_date":"2023-11-18"},{"userId":1,"calendar_title":"3\uc8fc \ud68c\uc2dd","calendar_place":"\uc11c\uba74","calendar_start_date":1700265600000,"calendar_end_date":"2023-11-18"}]이번 주 토요일 일정은 다음과 같습니다:

```json
[
  {
    "userId": 1,
    "calendar_title": "3주 회식",
    "calendar_place": "서면",
    "calendar_start_date": 1700265600000,
    "calendar_end_date": "2023-11-18"
  },
  {
    "userId": 1,
    "calendar_title": "3주 회식",
    "calendar

In [6]:
print(res)

이번 주 토요일 일정은 다음과 같습니다:

```json
[
  {
    "userId": 1,
    "calendar_title": "3주 회식",
    "calendar_place": "서면",
    "calendar_start_date": 1700265600000,
    "calendar_end_date": "2023-11-18"
  },
  {
    "userId": 1,
    "calendar_title": "3주 회식",
    "calendar_place": "서면",
    "calendar_start_date": 1700265600000,
    "calendar_end_date": "2023-11-18"
  }
]
```

위 결과는 JSON 형식으로 출력되었습니다.


# 날짜만 변환해서 직접 찾기

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain
from langchain.schema import HumanMessage

def to_datetime(inp):
    
    chat = ChatOpenAI()
    output = chat(
            [
                HumanMessage(
                    content=f"""
                        Convert the phrase "{inp}" to a date in YYYY-MM-DD format, considering today as the reference.
                        If today's date is 2023-11-14,
                        
                        Input: 오늘 
                        Output: 2023-11-14
                        
                        Input: 이번 주 일요일  
                        Output: 2023-11-19
                        
                        Input: 다움 주 월요일  
                        Output: 2023-11-20
                        
                        Input: 이번 주 
                        Output:2023-11-14, 2023-11-15, 2023-11-16, 2023-11-17, 2023-11-18, 2023-11-19, 2023-11-20
                        
                        Input: 다음 주 
                        Output:2023-11-20, 2023-11-21, 2023-11-22, 2023-11-23, 2023-11-24, 2023-11-25, 2023-11-26
                        
                        Input: {inp}
                        Output:"""
                                )
            ]
    )
    
    return output.content

In [15]:
date_res = to_datetime('토요일 일정 알려줘')

In [18]:
date_list = date_res.split(', ')

In [22]:
import pandas as pd

# CSV 파일 경로
csv_file_path = "../data/cal_1.csv"

# CSV 파일을 DataFrame으로 읽기
df = pd.read_csv(csv_file_path)

# cal_start_date가 target_dates에 있는 행만 추출
filtered_df = df[df['calendar_start_date'].isin(date_list)]

# 결과 출력
print(filtered_df)


   userId calendar_title calendar_place calendar_start_date calendar_end_date
0       1          3주 회식             서면          2023-11-18        2023-11-18
1       1          3주 회식             서면          2023-11-18        2023-11-18
